In [1]:
# Import Tecton and other libraries
import logging
import os
import tecton
from dotenv import load_dotenv
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta
from pprint import pprint

load_dotenv()  # take environment variables from .env.
logging.getLogger('snowflake.connector').setLevel(logging.WARNING)
logging.getLogger('snowflake.snowpark').setLevel(logging.WARNING)

connection_parameters = {
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_PASSWORD'],
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "warehouse": "DEMO_WH",
    # Database and schema are required to create various temporary objects by tecton
    "database": "TECTON",
    "schema": "PUBLIC",
}
conn = snowflake.connector.connect(**connection_parameters)
tecton.snowflake_context.set_connection(conn) # Tecton will use this Snowflake connection for all interactive queries


# Quick helper function to query snowflake from a notebook
# Make sure to replace with the appropriate connection details for your own account
def query_snowflake(query):
    df = conn.cursor().execute(query).fetch_pandas_all()
    return df

# Generate Inference Features with Tecton

In [2]:
ws = tecton.get_workspace('apply-2022-demo')
fs = ws.get_feature_service('batch_movie_recommendations_feature_service')

In [3]:
spine_full = """
    with 
      spine_movies as 
        (select movie_id from
          (select
              movie_id,
              count(rating) as num_ratings
          from
              (select * from DEV_DAVID.MOVIELENS_25M.RATINGS where timestamp > '2021-05-01 00:00:01')
          group by
              movie_id)
        WHERE num_ratings > 100),
      spine_users as 
          (select distinct user_id from (select user_id, timestamp from DEV_DAVID.MOVIELENS_25M.RATINGS where timestamp > '2022-04-01 00:00:01'))
    select
        user_id,
        movie_id,
        CURRENT_TIMESTAMP as TIMESTAMP
    from
        spine_movies cross join spine_users
"""

In [4]:
inference_spine = fs.get_historical_features(spine=spine_full, timestamp_key="TIMESTAMP").to_pandas()
inference_spine.head(10)

,MOVIE_ID,USER_ID,TIMESTAMP,MOVIE_GENRE_INFO__IS_ACTION,MOVIE_GENRE_INFO__IS_ANIMATION,MOVIE_GENRE_INFO__IS_ADVENTURE,MOVIE_GENRE_INFO__IS_CHILDREN,MOVIE_GENRE_INFO__IS_COMEDY,MOVIE_GENRE_INFO__IS_CRIME,MOVIE_GENRE_INFO__IS_DOCUMENTARY,...,USER_FANTASY_RATING_HISTORY__RATING_MEAN_730D_1D,USER_FILM_NOIR_RATING_HISTORY__RATING_MEAN_730D_1D,USER_HORROR_RATING_HISTORY__RATING_MEAN_730D_1D,USER_MUSICAL_RATING_HISTORY__RATING_MEAN_730D_1D,USER_MYSTERY_RATING_HISTORY__RATING_MEAN_730D_1D,USER_ROMANCE_RATING_HISTORY__RATING_MEAN_730D_1D,USER_SCI_FI_RATING_HISTORY__RATING_MEAN_730D_1D,USER_THRILLER_RATING_HISTORY__RATING_MEAN_730D_1D,USER_WAR_RATING_HISTORY__RATING_MEAN_730D_1D,USER_WESTERN_RATING_HISTORY__RATING_MEAN_730D_1D
0,1625,124862,2022-05-06 07:52:59.344000-07:00,0,0,0,0,0,0,0,...,4.043478000000,None,3.833333000000,4.000000000000,3.900000000000,4.136364000000,3.871795000000,4.161290000000,4.000000000000,4.000000000000
1,36529,6776,2022-05-06 07:52:59.344000-07:00,1,0,0,0,0,1,0,...,3.500000000000,None,3.250000000000,2.666666666667,2.800000000000,2.904761904762,3.000000000000,3.000000000000,3.000000000000,3.000000000000
2,595,83376,2022-05-06 07:52:59.344000-07:00,0,1,0,1,0,0,0,...,4.428571428571,None,None,None,None,5.000000000000,4.333333333333,4.000000000000,None,None
3,80846,112441,2022-05-06 07:52:59.344000-07:00,0,0,0,0,0,0,0,...,2.982456157895,None,2.100000000000,2.818181818182,3.370370296296,3.175675594595,2.999999986842,3.221153875000,2.916666750000,3.142857142857
4,1240,29004,2022-05-06 07:52:59.344000-07:00,1,0,0,0,0,0,0,...,3.250000250000,None,3.750000000000,None,4.000000000000,3.000000000000,3.421052684211,3.714286000000,4.000000000000,4.000000000000
5,5219,19997,2022-05-06 07:52:59.344000-07:00,1,0,0,0,0,0,0,...,4.108695673913,4.785714285714,4.076923038462,3.777777777778,4.701492522388,3.924999800000,4.297029861386,4.499999923077,4.341463439024,4.000000076923
6,1240,92565,2022-05-06 07:52:59.344000-07:00,1,0,0,0,0,0,0,...,2.500000000000,None,2.000000000000,None,2.000000000000,4.000000000000,2.866666666667,2.103448275862,2.333333333333,2.000000000000
7,1625,51734,2022-05-06 07:52:59.344000-07:00,0,0,0,0,0,0,0,...,3.348484863636,2.500000000000,3.666666666667,3.999999750000,3.105263210526,3.642857257143,3.043478434783,3.206896551724,3.714285714286,4.666666666667
8,1625,60905,2022-05-06 07:52:59.344000-07:00,0,0,0,0,0,0,0,...,3.619047619048,4.500000000000,3.560000080000,3.333333333333,3.999999960000,3.882352882353,3.500000033333,3.759259314815,3.900000000000,3.750000000000
9,84392,67604,2022-05-06 07:52:59.344000-07:00,0,0,0,0,0,1,0,...,3.217391173913,4.428571285714,3.133333000000,2.249999750000,4.444444305556,3.471697886792,3.717948871795,4.059524023810,3.944444388889,4.500000250000


In [14]:
inference_spine.to_parquet('inference_data/movies_inference_5_9.pq')

# Execute Batch Inference

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
inference_data = pd.read_parquet('inference_data/movies_inference_5_6.pq').fillna(-1.)
for col in inference_data.columns:
    if "RATING_HISTORY" in col:
        inference_data[col] = inference_data[col].astype('float64')

In [6]:
from preprocess import preprocess_for_tabnet
x,_,_,_ = preprocess_for_tabnet(inference_data, train=False)
pred_movies = list(inference_data.MOVIE_ID.values)
pred_users = list(inference_data.USER_ID.values)

In [10]:
model_path = "models/batch_movie_recommender_5_9.zip"

loaded_clf = TabNetRegressor()
loaded_clf.load_model(model_path)

Device used : cpu
Device used : cpu


In [11]:
preds = loaded_clf.predict(x.values).flatten().tolist()

# Load Predictions into Snowflake

In [13]:
import snowflake.connector
from dotenv import load_dotenv
import os
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime, timedelta

load_dotenv()  # take environment variables from .env.

connection_parameters = {
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_PASSWORD'],
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "warehouse": "DEMO_WH",
    # Database and schema are required to create various temporary objects by tecton
    "database": "TECTON",
    "schema": "PUBLIC",
}
conn = snowflake.connector.connect(**connection_parameters)

In [14]:
pred_df = pd.DataFrame(
    {
        'USER_ID': pred_users,
        'MOVIE_ID': pred_movies,
        'PREDICTED_RATING': preds,
        'CREATED_AT': [datetime.now()-timedelta(days=1)]*len(preds)
    }
)
pred_df

,USER_ID,MOVIE_ID,PREDICTED_RATING,CREATED_AT
0,3128,15354,4.257931,2022-05-08 21:36:50.326400
1,14788,37080,3.132670,2022-05-08 21:36:50.326400
2,16843,41468,4.486698,2022-05-08 21:36:50.326400
3,1587,45876,2.861315,2022-05-08 21:36:50.326400
4,9756,4654,3.960298,2022-05-08 21:36:50.326400
...,...,...,...,...
5959502,17853,6343,3.425151,2022-05-08 21:36:50.326400
5959503,11348,31136,3.450816,2022-05-08 21:36:50.326400
5959504,2524,37280,4.325292,2022-05-08 21:36:50.326400
5959505,8509,16652,4.068108,2022-05-08 21:36:50.326400


In [15]:
write_pandas(conn, pred_df, "USER_PREDICTED_RATINGS", database='DEV_DAVID', schema="MOVIELENS_25M")

(True,
 1,
 5959507,
 [('zpwqo/file0.txt',
   'LOADED',
   5959507,
   5959507,
   1,
   0,
   None,
   None,
   None,
   None)])